In [1]:
import re
import json
import numpy as np
import pandas as pd
import jieba
from zhconv import convert
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

In [2]:
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_67195/3330071591.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [3]:
all_data.columns

Index(['cv_id', 'jd_id', 'jd_code', 'title', 'deadline', 'category_id',
       'category_name', 'company_id', 'company_name', 'recruit_number',
       'work_age', 'min_annual_salary', 'max_annual_salary',
       'min_month_salary', 'max_month_salary', 'pay_months', 'created_by_name',
       'created_by', 'created_at', 'updated_by', 'updated_at', 'tenant_id',
       'guarantee_period', 'status', 'recommended_rcn', 'tags', 'workplaces',
       'description', 'school', 'degree', 'post_level_id', 'requirement', 'ds',
       '_id', 'additionInfo', 'age', 'certificates', 'code', 'competenceScore',
       'completeness', 'countryCode', 'createdAt', 'createdBy',
       'currentCompany', 'currentPosition', 'currentSalary', 'dateOfBirth',
       'degree.1', 'desiredIndustry', 'desiredPosition', 'desiredSalary',
       'dutyTime', 'expectLocation', 'gender', 'importType', 'industry',
       'interviewTime', 'jobIntentStatus', 'languageSkills', 'languages',
       'location', 'maintainerId', 'majo

In [4]:
all_data[['work_age', 'workYear']]

,work_age,workYear
0,5-10年,13.0
1,3-5年,7.0
2,10年以上,14.0
3,3-5年,3.0
4,3-5年,5.0
...,...,...
81291,5-10年,1.0
81292,10年以上,23.0
81293,10年以上,23.0
81294,3-5年,5.0


## 理清需要用的特征


In [5]:
# 这些特征在sample pipeline的表里，但是现在无法下载
ext_columns = ['school_labels_cv', 'school_labels_jd', 'expectlocation_range_cv', 'location_range_jd', 'degree_index_cv', 'degree_index_jd', 'workyear_range_jd', 'gender_index_cv', 'job_intentstatus_cv']

# 这些特征在原始的cv和jd表里
raw_columns = ['age', 'recruit_number', 'workYear', 'min_annual_salary', 'max_annual_salary', 'pay_months', 'currentSalary', 'desiredSalary']

# 这些特征的过程已经有了，需要进一步拼接
new_columns = ['position_tfidf_pca', 'skills_tfidf_pca', 'jobTracks_tfidf_pca', 'title_category_tags_tfidf_pca', 'description_tfidf_pca', 'requirement_tfidf_pca', 'work_duration_mean', 'equal_words', 'equal_job']

# 将以上的特征合理分散在wide和deep侧
wide_columns = []
deep_columns = []

In [6]:
all_data[['cv_id', 'jd_id']+raw_columns]

,cv_id,jd_id,age,recruit_number,workYear,min_annual_salary,max_annual_salary,pay_months,currentSalary,desiredSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,36.0,1,13.0,80.0,150.0,NaN,NaN,NaN
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,29.0,1,7.0,28.0,35.0,NaN,NaN,NaN
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,NaN,1,14.0,25.0,35.0,NaN,NaN,NaN
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,27.0,1,3.0,60.0,500.0,NaN,NaN,NaN
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,28.0,1,5.0,NaN,NaN,12.0,保密,25 - 30k · 15薪
...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,41.0,1,1.0,NaN,NaN,12.0,6000,6千-8千/月
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,47.0,1,23.0,NaN,NaN,12.0,60+期权,100
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,47.0,1,23.0,NaN,NaN,13.0,60+期权,100
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,30.0,30,5.0,NaN,NaN,14.0,25K,28K


## tfidf_pca

In [18]:
import pandas as pd
# 读入20220821所有数据
all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_67195/358587841.py:3: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  all_data = pd.read_csv('../data/all_sample_20220821_spark.csv').drop(['Unnamed: 0'], axis=1)


In [23]:
sample = all_data

# 需要将这个向量变成一个列表
def tfidf_pca_merge_fun(series):
    return list(series)

tfidf_pca_file = ['description_tfidf_pca.csv', 'position_tfidf_pca.csv', 'requirement_tfidf_pca.csv', 'skills_tfidf_pca.csv', 'title_category_tags_tfidf_pca.csv']
for csv_file in tfidf_pca_file:
    tfidf_pca = pd.read_csv('../data/'+csv_file).drop(['Unnamed: 0'], axis=1)
    print(csv_file)
    #print(tfidf_pca)
    col_name = csv_file.strip('.csv') # 把skills的s去掉了
    tfidf_pca[col_name] = tfidf_pca.apply(tfidf_pca_merge_fun, axis=1)
    sample = pd.concat([sample, tfidf_pca[[col_name]]], axis=1)

tfidf_pca_col_names = [csv_file.strip('.csv') for csv_file in tfidf_pca_file]
sample[['cv_id', 'jd_id']+tfidf_pca_col_names]

description_tfidf_pca.csv
position_tfidf_pca.csv
requirement_tfidf_pca.csv
skills_tfidf_pca.csv
title_category_tags_tfidf_pca.csv


,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062349781737, 0.0482743680202122, 0.06...","[-0.0393278199442298, -0.1156517704532679, -0....","[-0.0884550034452465, -0.1255198333761071, -0....","[0.3299248866863066, 0.1647421092365002, 0.188...","[-0.0428003203409762, -0.1584213737525012, -0...."
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193590974826, 0.0553739306045292, -0.1...","[0.105418076631442, -0.0263972096958627, -0.07...","[-0.0803609388724612, -0.1239125127626652, -0....","[0.2365059778721167, 0.1136817584888073, -0.23...","[-0.2368437713677324, 0.0858238570289931, -0.2..."
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212102320653, -0.1819505434089687, -0....","[-0.0368150744816855, -0.08686207712637, -0.05...","[-0.0644952415511043, -0.1617720006466659, -0....","[-0.0915394250200466, -0.1409955414961084, -0....","[-0.1908741311485324, 0.1941299966047448, 0.34..."
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188854096615, 0.0989578351223518, -0.0...","[0.0490419846331619, -0.0412213766488918, -0.0...","[-0.1351532044421416, 0.1740610172368603, 0.04...","[0.1429876459372993, 0.0219407450614701, -0.14...","[-0.3720984118629293, 0.1912181895192234, -0.3..."
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442610032263, -0.033007167462663, 0.064...","[-0.0271020069566911, -0.1132012426944732, -0....","[-0.0138174208622499, 0.1003273895430437, -0.0...","[-0.0992961447872781, -0.1495964567985471, -0....","[0.0056217413242851, -0.0571649883703878, 0.00..."
...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.0614540099630116, -0.0867970373293865, -0....","[-0.0745961614271238, -0.0529118709389064, -0....","[-0.0400398537565921, -0.0616277680501727, 0.0...","[-0.0986273365526512, -0.182001001137766, -0.0...","[0.0344861406832975, -0.0468051111890295, 0.05..."
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[0.0151640640236356, -0.0802996065385363, 0.00..."
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[-0.0053327070703121, -0.0845578129629915, 0.0..."
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.407429499596706, -0.0564815654402401, 0.382...","[-0.3849339451202287, 0.4677351575623144, 0.34...","[0.5339787830860457, 0.078352462761449, 0.1293...","[-0.2529650801386044, 0.1589143739954893, 0.08...","[0.5471976342215813, 0.4049098503344869, -0.15..."


## work_duration

In [24]:
work_duration_mean_df = pd.read_csv('../data/work_duration_mean.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, work_duration_mean_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062349781737, 0.0482743680202122, 0.06...","[-0.0393278199442298, -0.1156517704532679, -0....","[-0.0884550034452465, -0.1255198333761071, -0....","[0.3299248866863066, 0.1647421092365002, 0.188...","[-0.0428003203409762, -0.1584213737525012, -0....",1209.500000
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193590974826, 0.0553739306045292, -0.1...","[0.105418076631442, -0.0263972096958627, -0.07...","[-0.0803609388724612, -0.1239125127626652, -0....","[0.2365059778721167, 0.1136817584888073, -0.23...","[-0.2368437713677324, 0.0858238570289931, -0.2...",506.000000
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212102320653, -0.1819505434089687, -0....","[-0.0368150744816855, -0.08686207712637, -0.05...","[-0.0644952415511043, -0.1617720006466659, -0....","[-0.0915394250200466, -0.1409955414961084, -0....","[-0.1908741311485324, 0.1941299966047448, 0.34...",811.500000
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188854096615, 0.0989578351223518, -0.0...","[0.0490419846331619, -0.0412213766488918, -0.0...","[-0.1351532044421416, 0.1740610172368603, 0.04...","[0.1429876459372993, 0.0219407450614701, -0.14...","[-0.3720984118629293, 0.1912181895192234, -0.3...",76.500000
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442610032263, -0.033007167462663, 0.064...","[-0.0271020069566911, -0.1132012426944732, -0....","[-0.0138174208622499, 0.1003273895430437, -0.0...","[-0.0992961447872781, -0.1495964567985471, -0....","[0.0056217413242851, -0.0571649883703878, 0.00...",1034.000000
...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.0614540099630116, -0.0867970373293865, -0....","[-0.0745961614271238, -0.0529118709389064, -0....","[-0.0400398537565921, -0.0616277680501727, 0.0...","[-0.0986273365526512, -0.182001001137766, -0.0...","[0.0344861406832975, -0.0468051111890295, 0.05...",91.000000
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[0.0151640640236356, -0.0802996065385363, 0.00...",1234.857143
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[-0.0053327070703121, -0.0845578129629915, 0.0...",1234.857143
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.407429499596706, -0.0564815654402401, 0.382...","[-0.3849339451202287, 0.4677351575623144, 0.34...","[0.5339787830860457, 0.078352462761449, 0.1293...","[-0.2529650801386044, 0.1589143739954893, 0.08...","[0.5471976342215813, 0.4049098503344869, -0.15...",598.666667


## equal_words

In [25]:
equal_job_df = pd.read_csv('../data/equal_job.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, equal_job_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062349781737, 0.0482743680202122, 0.06...","[-0.0393278199442298, -0.1156517704532679, -0....","[-0.0884550034452465, -0.1255198333761071, -0....","[0.3299248866863066, 0.1647421092365002, 0.188...","[-0.0428003203409762, -0.1584213737525012, -0....",1209.500000,3
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193590974826, 0.0553739306045292, -0.1...","[0.105418076631442, -0.0263972096958627, -0.07...","[-0.0803609388724612, -0.1239125127626652, -0....","[0.2365059778721167, 0.1136817584888073, -0.23...","[-0.2368437713677324, 0.0858238570289931, -0.2...",506.000000,4
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212102320653, -0.1819505434089687, -0....","[-0.0368150744816855, -0.08686207712637, -0.05...","[-0.0644952415511043, -0.1617720006466659, -0....","[-0.0915394250200466, -0.1409955414961084, -0....","[-0.1908741311485324, 0.1941299966047448, 0.34...",811.500000,0
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188854096615, 0.0989578351223518, -0.0...","[0.0490419846331619, -0.0412213766488918, -0.0...","[-0.1351532044421416, 0.1740610172368603, 0.04...","[0.1429876459372993, 0.0219407450614701, -0.14...","[-0.3720984118629293, 0.1912181895192234, -0.3...",76.500000,3
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442610032263, -0.033007167462663, 0.064...","[-0.0271020069566911, -0.1132012426944732, -0....","[-0.0138174208622499, 0.1003273895430437, -0.0...","[-0.0992961447872781, -0.1495964567985471, -0....","[0.0056217413242851, -0.0571649883703878, 0.00...",1034.000000,1
...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.0614540099630116, -0.0867970373293865, -0....","[-0.0745961614271238, -0.0529118709389064, -0....","[-0.0400398537565921, -0.0616277680501727, 0.0...","[-0.0986273365526512, -0.182001001137766, -0.0...","[0.0344861406832975, -0.0468051111890295, 0.05...",91.000000,2
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[0.0151640640236356, -0.0802996065385363, 0.00...",1234.857143,3
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[-0.0053327070703121, -0.0845578129629915, 0.0...",1234.857143,4
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.407429499596706, -0.0564815654402401, 0.382...","[-0.3849339451202287, 0.4677351575623144, 0.34...","[0.5339787830860457, 0.078352462761449, 0.1293...","[-0.2529650801386044, 0.1589143739954893, 0.08...","[0.5471976342215813, 0.4049098503344869, -0.15...",598.666667,5


## salary

In [26]:
salary_df = pd.read_csv('../data/salary.csv').drop(['Unnamed: 0'], axis=1)

sample = pd.concat([sample, salary_df], axis=1)

columns = ['cv_id', 'jd_id'] + tfidf_pca_col_names + ['work_duration_mean'] + ['equal_job'] + ['parsed_desiredSalary', 'parsed_currentSalary']
sample[columns]

,cv_id,jd_id,description_tfidf_pca,position_tfidf_pca,requirement_tfidf_pca,kills_tfidf_pca,title_category_tags_tfidf_pca,work_duration_mean,equal_job,parsed_desiredSalary,parsed_currentSalary
0,2c9207157bf91042017c0d2beef00470,2c9207157bcbb76b017bcde2b46d04cd,"[-0.2036062349781737, 0.0482743680202122, 0.06...","[-0.0393278199442298, -0.1156517704532679, -0....","[-0.0884550034452465, -0.1255198333761071, -0....","[0.3299248866863066, 0.1647421092365002, 0.188...","[-0.0428003203409762, -0.1584213737525012, -0....",1209.500000,3,-1,-1
1,2c9207157bf91042017c0d979cbb047d,2c9207157bcbb76b017bf7f2bbbb3153,"[-0.0922193590974826, 0.0553739306045292, -0.1...","[0.105418076631442, -0.0263972096958627, -0.07...","[-0.0803609388724612, -0.1239125127626652, -0....","[0.2365059778721167, 0.1136817584888073, -0.23...","[-0.2368437713677324, 0.0858238570289931, -0.2...",506.000000,4,-1,-1
2,2c9207157bf91042017c7232fbab3be1,2c9207157bf94d99017c108f769405e0,"[-0.0731212102320653, -0.1819505434089687, -0....","[-0.0368150744816855, -0.08686207712637, -0.05...","[-0.0644952415511043, -0.1617720006466659, -0....","[-0.0915394250200466, -0.1409955414961084, -0....","[-0.1908741311485324, 0.1941299966047448, 0.34...",811.500000,0,-1,-1
3,2c9207157d308592017d4a94cb1113a1,2c9207157d2f0ec5017d5a9265ba33d9,"[-0.0941188854096615, 0.0989578351223518, -0.0...","[0.0490419846331619, -0.0412213766488918, -0.0...","[-0.1351532044421416, 0.1740610172368603, 0.04...","[0.1429876459372993, 0.0219407450614701, -0.14...","[-0.3720984118629293, 0.1912181895192234, -0.3...",76.500000,3,-1,-1
4,2c9207157dabb0a3017dbca16cdf4c26,2c9207157d631abb017d6eed1c79139f,"[0.0394442610032263, -0.033007167462663, 0.064...","[-0.0271020069566911, -0.1132012426944732, -0....","[-0.0138174208622499, 0.1003273895430437, -0.0...","[-0.0992961447872781, -0.1495964567985471, -0....","[0.0056217413242851, -0.0571649883703878, 0.00...",1034.000000,1,450,-1
...,...,...,...,...,...,...,...,...,...,...,...
81291,8a69d7c28219945301823d8a08d14e70,8a69f6f8823a93b901823d891b761d51,"[-0.0614540099630116, -0.0867970373293865, -0....","[-0.0745961614271238, -0.0529118709389064, -0....","[-0.0400398537565921, -0.0616277680501727, 0.0...","[-0.0986273365526512, -0.182001001137766, -0.0...","[0.0344861406832975, -0.0468051111890295, 0.05...",91.000000,2,-1,-1
81292,8a69d7c282199453018247bac4a36859,8a69c468826ee9b40182831bedfc5e79,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[0.0151640640236356, -0.0802996065385363, 0.00...",1234.857143,3,-1,-1
81293,8a69d7c282199453018247bac4a36859,8a69f6f8823a93b9018247c30aca4e3d,"[-0.1197626282544053, 0.2009878664368261, 0.10...","[-0.0248770953309609, -0.0818758200524019, -0....","[-0.1178366574501578, 0.1256630104745626, 0.14...","[0.0156514140907682, 0.0238283438685977, -0.06...","[-0.0053327070703121, -0.0845578129629915, 0.0...",1234.857143,4,-1,-1
81294,8a69d7db826df6c201827b4338cd60c2,2c92071580dd3ab70180f18714e119c0,"[0.407429499596706, -0.0564815654402401, 0.382...","[-0.3849339451202287, 0.4677351575623144, 0.34...","[0.5339787830860457, 0.078352462761449, 0.1293...","[-0.2529650801386044, 0.1589143739954893, 0.08...","[0.5471976342215813, 0.4049098503344869, -0.15...",598.666667,5,336,300


## 数据集划分，剔除复推集